In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ../..

/media/samuele/Disco/PycharmProjectsUbuntu/MARL-CER


In [2]:
import jax
import jax.numpy as jnp
from flax import nnx
from functools import partial
from jax_tqdm import scan_tqdm

from algorithms.utils import save_state_multiagent

import numpy as np

# jax.config.update("jax_debug_nans", True)

2025-03-25 14:10:20.334329: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742908220.348745   69168 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742908220.352983   69168 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from ernestogym.envs_jax.multi_agent.env import RECEnv

In [4]:
def my_env_creator(params, battery_type, env_type='normal'):
    if env_type == 'normal':
        env = RECEnv(params, battery_type)
    return env

In [5]:
from ernestogym.envs_jax.multi_agent.utils import parameter_generator

In [6]:
# pack_options = "ernestogym/ernesto_jax/data/battery_new/pack.yaml"
# pack_options = "ernestogym/ernesto_jax/data/battery_new/pack_cheap.yaml"
# pack_options = "ernestogym/ernesto_jax/data/battery_new/pack_init_full.yaml"
# pack_options = "ernestogym/ernesto_jax/data/battery_new/pack_init_full_cheap.yaml"
pack_options = "ernestogym/ernesto_jax/data/battery_new/pack_init_half_full_cheap.yaml"

ecm = "ernestogym/ernesto_jax/data/battery_new/models/electrical/thevenin_pack.yaml"
# ecm = "ernestogym/ernesto_jax/data/battery_new/models/electrical/thevenin_fading_pack.yaml"
r2c = "ernestogym/ernesto_jax/data/battery_new/models/thermal/r2c_thermal_pack.yaml"
# bolun = "ernestogym/ernesto_jax/data/battery_new/models/aging/bolun_pack.yaml"
bolun = "ernestogym/ernesto_jax/data/battery_new/models/aging/bolun_pack.yaml"


# world = "ernestogym/envs_jax/multi_agent/world_deg_train.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_train_exp_avg.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_train_split_demands_exp_avg.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_train_split_generations.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_train_split_generations_exp_avg_passive.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_train_split_generations_exp_avg.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_train_split_generations_only_trad_exp_avg.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_train_all_the_same_debug.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_train_all_the_same_debug_like_single.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_train_split_generations_new_clip.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_train_split_generations_exp_avg_new_clip.yaml"

# world = "ernestogym/envs_jax/multi_agent/world_deg_train_split_generations_for_recurrent.yaml"
world = 'ernestogym/envs_jax/multi_agent/world_deg_train_split_generations_for_recurrent_passive_no_gen.yaml'
# world = "ernestogym/envs_jax/multi_agent/world_deg_train_split_generations_for_recurrent_passive_scaled.yaml"

# battery_type = 'fading'
# battery_type = 'degrading'
battery_type = 'degrading_dropflow'

params = parameter_generator(
    input_var='current',
    battery_options=pack_options,
    electrical_model=ecm,
    thermal_model=r2c,
    aging_model=bolun,
    world_options=world,
    # use_reward_normalization=True

)

params

yaaaas2


{'num_battery_agents': 3,
 'num_passive_houses': 0,
 'batteries': [{'sign_convention': 'passive',
   'params': {'nominal_voltage': 350.4,
    'nominal_capacity': 60.0,
    'nominal_dod': 0.8,
    'nominal_lifetime': 3000,
    'v_max': 398.4,
    'v_min': 288.0,
    'temp_ambient': 298.15,
    'nominal_cost': 1500.0},
   'bounds': {'voltage': {'low': 288.0, 'high': 398.4},
    'current': {'low': -180.0, 'high': 60.0},
    'power': {'low': -71712.0, 'high': 23904.0},
    'temperature': {'low': 273.15, 'high': 323.15},
    'temp_ambient': {'low': 273.15, 'high': 323.15},
    'soc': {'low': 0.2, 'high': 1.0},
    'soh': {'low': 0.6, 'high': 1.0}},
   'init': {'voltage': 310.8,
    'current': 0.0,
    'power': 0.0,
    'temperature': 293.15,
    'temp_ambient': 293.15,
    'soc': 0.5,
    'soh': 1.0}},
  {'sign_convention': 'passive',
   'params': {'nominal_voltage': 350.4,
    'nominal_capacity': 60.0,
    'nominal_dod': 0.8,
    'nominal_lifetime': 3000,
    'v_max': 398.4,
    'v_min': 2

In [7]:
# pack_options = "ernestogym/ernesto_jax/data/battery_new/pack.yaml"
# pack_options = "ernestogym/ernesto_jax/data/battery_new/pack_cheap.yaml"
# pack_options = "ernestogym/ernesto_jax/data/battery_new/pack_init_full.yaml"
# pack_options = "ernestogym/ernesto_jax/data/battery_new/pack_init_full_cheap.yaml"
pack_options = "ernestogym/ernesto_jax/data/battery_new/pack_init_half_full_cheap.yaml"

ecm = "ernestogym/ernesto_jax/data/battery_new/models/electrical/thevenin_pack.yaml"
# ecm = "ernestogym/ernesto_jax/data/battery_new/models/electrical/thevenin_fading_pack.yaml"
r2c = "ernestogym/ernesto_jax/data/battery_new/models/thermal/r2c_thermal_pack.yaml"
# bolun = "ernestogym/ernesto_jax/data/battery_new/models/aging/bolun_pack.yaml"
bolun = "ernestogym/ernesto_jax/data/battery_new/models/aging/bolun_pack.yaml"



# world = "ernestogym/envs_jax/multi_agent/world_deg_test.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_test_exp_avg.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_test_split_demands_exp_avg.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_test_split_generations.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_test_split_generations_exp_avg_passive.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_test_split_generations_exp_avg.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_test_split_generations_only_trad_exp_avg.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_test_all_the_same_debug.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_test_all_the_same_debug_like_single.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_test_split_generations_new_clip.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_test_split_generations_exp_avg_new_clip.yaml"

# world = "ernestogym/envs_jax/multi_agent/world_deg_test_split_generations_for_recurrent.yaml"
world = 'ernestogym/envs_jax/multi_agent/world_deg_test_split_generations_for_recurrent_passive_no_gen.yaml'
# world = "ernestogym/envs_jax/multi_agent/world_deg_test_split_generations_for_recurrent_passive_scaled.yaml"

params_testing = parameter_generator(
    input_var='current',
    battery_options=pack_options,
    electrical_model=ecm,
    thermal_model=r2c,
    aging_model=bolun,
    world_options=world,
    # use_reward_normalization=True

)

params_testing['step']

yaaaas2


3600

In [8]:
from algorithms.multi_agent_ppo_only_rl import make_train, train_wrapper
from flax.core.frozen_dict import freeze, unfreeze

In [9]:
num_envs = 4
total_timesteps = 8760 * num_envs * 500

env_type = 'normal'

config = {
    "LR_BATTERIES": 1e-4,
    "LR_BATTERIES_MIN": 1e-6,
    'FRACTION_DYNAMIC_LR_BATTERIES': 0.7,
    "LR_REC": 1e-4,
    "LR_REC_MIN": 1e-6,
    'FRACTION_DYNAMIC_LR_REC': 0.7,
    "NUM_ENVS": num_envs,
    "NUM_STEPS": 8192, #2048 #10000,
    # "TOTAL_TIMESTEPS": 100000 * num_envs,
    "TOTAL_TIMESTEPS": total_timesteps,
    "UPDATE_EPOCHS": 10,
    "NUM_MINIBATCHES": 32, #64,
    "NORMALIZE_ENV": False,
    "GAMMA": 0.99,
    "GAE_LAMBDA": 0.98,
    "CLIP_EPS": 0.20,
    "ENT_COEF": 0., #0.01,
    "VF_COEF": 0.5,
    "MAX_GRAD_NORM": 0.5,
    "NET_ARCH": (64, 32, 16), #(16, 16, 16),
    "LSTM_NET_ARCH": (24, 12),
    "ACTIVATION": 'tanh',
    "LSTM_ACTIVATION": 'tanh',
    "LOGISTIC_FUNCTION_TO_ACTOR": False,
    "ENV_NAME": None,
    "LR_SCHEDULE": 'cosine',
    'DEBUG': False,
    "NORMALIZE_REWARD_FOR_GAE_AND_TARGETS": False,
    "NORMALIZE_TARGETS": False,
    "NORMALIZE_ADVANTAGES": True,
    "ENV_TYPE": env_type,
    'NETWORK_TYPE_BATTERIES': 'actor_critic',
    'NETWORK_TYPE_REC': 'actor_critic',
    'USE_WEIGHT_DECAY': True,
    'NORMALIZE_NN_INPUTS': True
}

rng = jax.random.PRNGKey(42)

env = my_env_creator(params, battery_type, env_type=env_type)
env, train_state = make_train(config, env)

env_testing = my_env_creator(params_testing, battery_type, env_type=env_type)

val_rng = jax.random.PRNGKey(51)
val_num_iters = 8670*5

# out = train_jit(rng)

config = freeze(config)

norm? False
norm batt
norm rec
norm? False


In [10]:
config

FrozenDict({
    LR_BATTERIES: 0.0001,
    LR_BATTERIES_MIN: 1e-06,
    FRACTION_DYNAMIC_LR_BATTERIES: 0.7,
    LR_REC: 0.0001,
    LR_REC_MIN: 1e-06,
    FRACTION_DYNAMIC_LR_REC: 0.7,
    NUM_ENVS: 4,
    NUM_STEPS: 8192,
    TOTAL_TIMESTEPS: 5256000,
    UPDATE_EPOCHS: 10,
    NUM_MINIBATCHES: 32,
    NORMALIZE_ENV: False,
    GAMMA: 0.99,
    GAE_LAMBDA: 0.98,
    CLIP_EPS: 0.2,
    ENT_COEF: 0.0,
    VF_COEF: 0.5,
    MAX_GRAD_NORM: 0.5,
    NET_ARCH: (64, 32, 16),
    LSTM_NET_ARCH: (24, 12),
    ACTIVATION: 'tanh',
    LSTM_ACTIVATION: 'tanh',
    LOGISTIC_FUNCTION_TO_ACTOR: False,
    ENV_NAME: None,
    LR_SCHEDULE: 'cosine',
    DEBUG: False,
    NORMALIZE_REWARD_FOR_GAE_AND_TARGETS: False,
    NORMALIZE_TARGETS: False,
    NORMALIZE_ADVANTAGES: True,
    ENV_TYPE: 'normal',
    NETWORK_TYPE_BATTERIES: 'actor_critic',
    NETWORK_TYPE_REC: 'actor_critic',
    USE_WEIGHT_DECAY: True,
    NORMALIZE_NN_INPUTS: True,
    NUM_UPDATES: 160,
    MINIBATCH_SIZE: 1024,
    BATTERY_ACTI

In [11]:
env.battery_agents

['battery_agent_0', 'battery_agent_1', 'battery_agent_2']

In [12]:
env.valorization_incentive_coeff

8e-06

In [13]:
networks_batteries, optimizer_batteries, network_rec, optimizer_rec = nnx.merge(train_state.graph_def, train_state.state)

In [14]:
optimizer_batteries

StackedOptimizer(
  model=StackedActorCritic(
    act_layers=[Linear(
      bias=Param(
        value=Array(shape=(3, 64), dtype=float32)
      ),
      bias_init=<function constant.<locals>.init at 0x78a910317880>,
      dot_general=<function dot_general at 0x78a995c57b00>,
      dtype=None,
      in_features=11,
      kernel=Param(
        value=Array(shape=(3, 11, 64), dtype=float32)
      ),
      kernel_init=<function orthogonal.<locals>.init at 0x78a910317920>,
      out_features=64,
      param_dtype=<class 'jax.numpy.float32'>,
      precision=None,
      use_bias=True
    ), <PjitFunction of <function tanh at 0x78a9958fa700>>, Linear(
      bias=Param(
        value=Array(shape=(3, 32), dtype=float32)
      ),
      bias_init=<function constant.<locals>.init at 0x78a9101ea0c0>,
      dot_general=<function dot_general at 0x78a995c57b00>,
      dtype=None,
      in_features=64,
      kernel=Param(
        value=Array(shape=(3, 64, 32), dtype=float32)
      ),
      kernel_init=<

In [15]:
networks_batteries

StackedActorCritic(
  act_layers=[Linear(
    bias=Param(
      value=Array(shape=(3, 64), dtype=float32)
    ),
    bias_init=<function constant.<locals>.init at 0x78a910317880>,
    dot_general=<function dot_general at 0x78a995c57b00>,
    dtype=None,
    in_features=11,
    kernel=Param(
      value=Array(shape=(3, 11, 64), dtype=float32)
    ),
    kernel_init=<function orthogonal.<locals>.init at 0x78a910317920>,
    out_features=64,
    param_dtype=<class 'jax.numpy.float32'>,
    precision=None,
    use_bias=True
  ), <PjitFunction of <function tanh at 0x78a9958fa700>>, Linear(
    bias=Param(
      value=Array(shape=(3, 32), dtype=float32)
    ),
    bias_init=<function constant.<locals>.init at 0x78a9101ea0c0>,
    dot_general=<function dot_general at 0x78a995c57b00>,
    dtype=None,
    in_features=64,
    kernel=Param(
      value=Array(shape=(3, 64, 32), dtype=float32)
    ),
    kernel_init=<function orthogonal.<locals>.init at 0x78a9101e9940>,
    out_features=32,
    par

In [16]:
network_rec

RECActorCritic(
  act_layers=[Linear(
    bias=Param(
      value=Array(shape=(64,), dtype=float32)
    ),
    bias_init=<function constant.<locals>.init at 0x78a910315440>,
    dot_general=<function dot_general at 0x78a995c57b00>,
    dtype=None,
    in_features=13,
    kernel=Param(
      value=Array(shape=(13, 64), dtype=float32)
    ),
    kernel_init=<function orthogonal.<locals>.init at 0x78a910316e80>,
    out_features=64,
    param_dtype=<class 'jax.numpy.float32'>,
    precision=None,
    use_bias=True
  ), <PjitFunction of <function tanh at 0x78a9958fa700>>, Linear(
    bias=Param(
      value=Array(shape=(32,), dtype=float32)
    ),
    bias_init=<function constant.<locals>.init at 0x78a9044df420>,
    dot_general=<function dot_general at 0x78a995c57b00>,
    dtype=None,
    in_features=64,
    kernel=Param(
      value=Array(shape=(64, 32), dtype=float32)
    ),
    kernel_init=<function orthogonal.<locals>.init at 0x78a9044df600>,
    out_features=32,
    param_dtype=<clas

In [ ]:
import time
rng = jax.random.PRNGKey(42)
t0 = time.time()

# with jax.checking_leaks():
out = train_wrapper(env, config, train_state, rng, validate=True, freq_val=10, val_env=env_testing, val_rng=val_rng, val_num_iters=val_num_iters, params=params, path_saving='/media/samuele/Disco/PycharmProjectsUbuntu/MARL-CER/trained_agents/')

train_state = out['runner_state'].train_state

print(f"time: {time.time() - t0:.2f} s")

Running for 160 iterations:   0%|          | 0/160 [00:00<?, ?it/s]

Running for 43,350 iterations:   0%|          | 0/43350 [00:00<?, ?it/s]

curr_iter: 0
	r_tot: -4038.8603515625
	r_trad: -2287.70849609375
	r_deg: -1963.242431640625
	r_clip: 0.0
	r_glob: 212.09042358398438
	r_rec: 1744596.5
	mean soc: 0.4225030541419983
	std actions: [0.01977567 0.03288947 0.03109958]
	self consumption: 1744596.5



Running for 43,350 iterations:   0%|          | 0/43350 [00:00<?, ?it/s]

curr_iter: 10
	r_tot: -3411.754150390625
	r_trad: -1829.35693359375
	r_deg: -1860.610107421875
	r_clip: 0.0
	r_glob: 278.21295166015625
	r_rec: 2300478.25
	mean soc: 0.29632365703582764
	std actions: [0.36103305 0.36238265 0.43437815]
	self consumption: 2300478.25



Running for 43,350 iterations:   0%|          | 0/43350 [00:00<?, ?it/s]

curr_iter: 20
	r_tot: -3132.7041015625
	r_trad: -1585.94140625
	r_deg: -1899.327880859375
	r_clip: 0.0
	r_glob: 352.56512451171875
	r_rec: 2917597.25
	mean soc: 0.2927018404006958
	std actions: [0.64290494 0.64707536 0.62782395]
	self consumption: 2917597.25



In [18]:
# jax.tree.map(lambda x: x.shape, out['metrics'])

In [19]:
# jax.tree.map(lambda x: jnp.isnan(x).any(), out['metrics'])

In [20]:
# jax.tree.map(lambda x: jnp.isnan(x).any(axis=jnp.arange(1, x.ndim)), out['metrics'])

In [21]:
# out['metrics']['actions']['REC_agent'][0]

In [22]:
# out['metrics']['actions']['REC_agent'][1]

In [23]:
# np.isnan(out['metrics']['actions']['REC_agent'][1]).any()

In [24]:
# jax.tree.map(lambda x: np.nonzero(np.isnan(x)), out['metrics'])

In [25]:
# out['metrics']['actions']['REC_agent']

In [26]:
# out['metrics']['pure_reward']['r_clipping'][3]

In [27]:
network_batteries, optimizer_batteries, network_rec, optimizer_rec = nnx.merge(train_state.graph_def, train_state.state)
network_batteries

StackedRecurrentActorCritic(
  act_layers=[Linear(
    bias=Param(
      value=Array(shape=(3, 64), dtype=float32)
    ),
    bias_init=<function constant.<locals>.init at 0x7fea1c3b56c0>,
    dot_general=<function dot_general at 0x7fead80f79c0>,
    dtype=None,
    in_features=16,
    kernel=Param(
      value=Array(shape=(3, 16, 64), dtype=float32)
    ),
    kernel_init=<function orthogonal.<locals>.init at 0x7fea1c3b5120>,
    out_features=64,
    param_dtype=<class 'jax.numpy.float32'>,
    precision=None,
    use_bias=True
  ), <PjitFunction of <function tanh at 0x7fead7daa5c0>>, Linear(
    bias=Param(
      value=Array(shape=(3, 32), dtype=float32)
    ),
    bias_init=<function constant.<locals>.init at 0x7fea1c326160>,
    dot_general=<function dot_general at 0x7fead80f79c0>,
    dtype=None,
    in_features=64,
    kernel=Param(
      value=Array(shape=(3, 64, 32), dtype=float32)
    ),
    kernel_init=<function orthogonal.<locals>.init at 0x7fea1c3259e0>,
    out_features=32